In [ ]:
# Classes are objects.
class A:
    def f(self):
        print('f', self)

def g(self):
    print('g', self)

a = A()
a.f()

a.g = g  # Add this as an object attribute.
try:
    a.g()  # Can't bind to self.
except Exception as e:
    print(e)

del a.g
A.g = g  # Add as a class attribute to bind to self.
a.g()

In [ ]:
# Miller Urey Experiment
class A:
    pass

def f(x):
    print(x)

# Functions are descriptors.
print('__get__' in dir(f))

a = A()
a.f = f
b = A()
b.f = f
print(a.f, b.f)

delattr(a, 'f')
A.f = f
print(a.f, b.f)  # Object attr takes precedence.

delattr(b, 'f')
delattr(A, 'f')
a.f = f.__get__(a, A)  # Object local version of A.f = f.
print(a.f)
try:
    print(b.f)
except Exception as e:
    print(e)

In [ ]:
# Implementing property as a descriptor.
class Property:
    def __init__(self, get, set=None):
        # Decoration on method allows captures it here.
        self._get = get
        self._set = set

    def __get__(self, instance, cls):
        # This is a descriptor, so calling the function name like a field
        # calls to here.
        if instance is None:
            # Called from class not instance `A.p`.
            return self
        return self._get(instance)

    def setter(self, fn):
        # Decorator that captures the behavior for __set__.
        self._set = fn

    def __set__(self, instance, value):
        if self._set is None:
            raise TypeError('No setter defined')
        if instance is None:
            # Called from class not instance `A.p`.
            return self
        return self._set(instance, value)

class A:
    @Property
    def p(self):
        return 1

    @p.setter
    def set(self, value):
        print(f'Set {value=}')

a = A()
print(a.p)
a.p = 2
A.p

In [13]:
# generators with 1 yield can be context managers. Use contextlib.

import time

def contextmanager(g):
    # g is a generator definition.

    class ContextManager:
        def __init__(self, *args, **kwargs):
            self.g = g(*args, **kwargs)

        def __enter__(self):
            return next(self.g)

        def __exit__(self, exception, error, traceback):
            if exception:
                self.execution.throw(exception, error, traceback)
                return

            try:
                next(self.g)
            except StopIteration:
                return

    return ContextManager

@contextmanager
def timer(x):
    print(f'{x=}')
    start = time.time()
    yield
    elapsed = time.time() - start
    print(f'elapsed: {elapsed:0.3f} seconds')

with timer(1):
    time.sleep(1)

x=1
elapsed: 1.001 seconds


In [22]:
# Context Manager  -- Due to raise, keep this as the last cell.
import time

class Timer:
    def __init__(self, suppress_errors=False):
        self.suppress_errors = suppress_errors 

    def __enter__(self):
        self.start = time.time()
        return self

    def __exit__(self, exception, error, traceback):
        elapsed = time.time() - self.start
        print(f'elapsed: {elapsed:0.3f} seconds')

        # Returning a value which evaluates to True will suppress errors.
        return self.suppress_errors

try:
    with Timer(suppress_errors=True):
        time.sleep(1)
        raise TypeError
except TypeError:
    print('Error')

elapsed: 1.001 seconds


In [19]:
# Object creation (not initialization).
class A:
    _cache = {}
    def __new__(cls, x):
        # Same signature as init, except self/cls.
        if x not in cls._cache:
            instance = super().__new__(cls)
            instance.x = x
            cls._cache[x] = instance

        # Calls __init__ after return.
        return cls._cache[x]
    
    def __init__(self, x):
        print('foo')

a1 = A(1)
a2 = A(1)
print(a1 is a2)  # Identical, not just equal.
a3 = A(3)
print(a1 is a3)



foo
foo
True
foo
False


In [ ]:
# Class decorators.
import threading

# Decorates all methods with synchronize.
def threadsafe(cls):
    cls._lock = threading.Lock()  # Class can't have _lock

    # Wrap all methods of the class.
    for key, value in cls.__dict___.items():
        if not callable(value):
            continue
        # Can't just use a lambda because that would cause late binding
        # of `value`.
        value = synchronize(value, cls._lock)
        setattr(cls, key, value)

    return cls

# Calls a function under lock.
def synchronize(function, lock):
    def wrapper(*args, **kwargs):
        with lock:
            return function(*args, **kwargs)
    return wrapper

In [34]:
# Dynamic classes
class A:
    for i in range(1):
        print('Creating A')

B = type('B', (A,), {'x': 1, 'f': lambda self: 42})
print(B)
b = B()
print(b.i, b.x)
b.f()


Creating A
<class '__main__.B'>
0 1


42

In [37]:
# Metaclasses - https://stackoverflow.com/questions/100003/what-are-metaclasses-in-python
class M(type):
    def __init__(cls, name, bases, attrs):
        print(name, bases, attrs)

class A(metaclass=M):
    pass

class B(A):
    x = 1
    def f(self):
        return 42

A () {'__module__': '__main__', '__qualname__': 'A'}
B (<class '__main__.A'>,) {'__module__': '__main__', '__qualname__': 'B', 'x': 1, 'f': <function B.f at 0x7ff7935b7430>}


In [47]:
# Type Safety - Metaclass v Decorator

class TypedProperty:
    def __init__(self, name, T):
        self.name = name
        self.T = T

    def __get__(self, instance, cls):
        if instance is None:
            return self
        if self.name not in instance.__dict__:
            instance.__dict__[self.name] = self.T()
        return  instance.__dict__[self.name]

    def __set__(self, instance, value):
        if not isinstance(value, self.T):
            raise TypeError(f'attribute {self.name} must by {self.T}')
        instance.__dict__[self.name] = value
        
class TypeSafe(type):
    def __init__(cls, name, bases, attrs):
        for key, value in attrs.items():
            if isinstance(value, type):
                value = TypedProperty(key, value)
                setattr(cls, key, value)

def typesafe(cls):
    for key, value in cls.__dict__.items():
        if isinstance(value, type):
            value = TypedProperty(key, value)
            setattr(cls, key, value)
    return cls

class A(metaclass=TypeSafe):
    x = int

@typesafe
class B:
    x = int

for cls in [A, B]:
    print(cls.__name__G
    )
    a = cls()
    print(a.x)
    a.x = 1
    print(a.x)

    try:
        a.x = '1'
    except TypeError as e:
        print(e)


A
0
1
attribute x must by <class 'int'>
B
0
1
attribute x must by <class 'int'>


In [56]:
# Enum - can only be done with metaclasses.
class ForgivingDict(dict):
    def __getitem__(self, key):
        print(key)
        if key not in self:
            self[key] = len(self)
        return super().__getitem__(key)

class EnumMetaclass(type):
    def __prepare__(metacls, cls):
        # This is the dict used to hold the namespace of subclasses.
        return ForgivingDict()

class Enum(metaclass=EnumMetaclass):
    # This allows users not to have to `metaclass`. This works because
    # metaclasses automatically work recursively on subclasses.
    pass

class Fruit(Enum):
    APPLE
    ORANGE
    BANANA

print(Fruit.APPLE, Fruit.ORANGE, Fruit.BANANA)

__name__
__name__
APPLE
ORANGE
BANANA
3 4 5
